In [1]:
import os
from tqdm import tqdm
import collections
import json
import copy

In [2]:
def list_files():
    files = os.listdir("danbooru/")
    return files

tagid2tagname = {}
tagname2tagid = {}

images = []

class image:
    def __init__(self, imid, imagetags):
        self.id = imid
        self.cat_tag = {
            0:[],
            1:[],
            2:[],
            3:[],
            4:[],
            5:[]
        }
        
        for tag in imagetags:
            self.cat_tag[int(tag['category'])].append(int(tag['id']))
            tagid2tagname[int(tag['id'])] = tag['name']
            tagname2tagid[tag['name']] = int(tag['id'])

def process_image(ind):
    a = json.loads(ind)
    if a['rating'] != 's':
        return False
    t = a['tags']
    
    newim = image(a['id'], t)
    for catgrp in newim.cat_tag.keys():
        for forbidden_tag in [358, 1936, 18, 29]:
            if forbidden_tag in newim.cat_tag[catgrp]:
                return False
            
    images.append(newim)
    return True
    
def succ_file(file):
    fc = 0
    trashed = 0
    with open(f"danbooru/{file}", "r") as f:
        for l in f.read().split("\n"):
            try:
                if not process_image(l):
                    trashed += 1
            except Exception as e:
                fc += 1
                print(e)
                
    print(f"wyjebano {fc + trashed}")

In [4]:
for file in tqdm(list(list_files())):
    succ_file(file)
print(len(images))

  6%|▌         | 1/17 [00:06<01:45,  6.57s/it]

Expecting value: line 1 column 1 (char 0)
wyjebano 86901


 12%|█▏        | 2/17 [00:13<01:39,  6.60s/it]

Expecting value: line 1 column 1 (char 0)
wyjebano 86911


 18%|█▊        | 3/17 [00:20<01:34,  6.75s/it]

...

Expecting value: line 1 column 1 (char 0)
wyjebano 86533


 94%|█████████▍| 16/17 [01:51<00:06,  6.94s/it]

Expecting value: line 1 column 1 (char 0)
wyjebano 86636


100%|██████████| 17/17 [01:57<00:00,  6.92s/it]

Expecting value: line 1 column 1 (char 0)
wyjebano 86709
1489727


In [5]:

def check_for_single_character(image):
    imtags = set(image.cat_tag[0])
    # multiple characters.

    # 1821 2girls
    if 1821 in imtags:
        return False
    # 449194 2boys
    if 449194 in imtags:
        return False
    
    # 399836 3girls
    if 399836 in imtags:
        return False
    #487156 3boys
    if 487156 in imtags:
        return False
    
    # 412555 4girls
    if 412555 in imtags:
        return False
    # 4boys 479955
    if 479955 in imtags:
        return False
    
    # 421662 5girls
    if 421662 in imtags:
        return False
    # 480899 5boys
    if 480899 in imtags:
        return False
    
    #567316 6+girls
    if 567316 in imtags:
        return False
    #572753 6+boys
    if 572753 in imtags:
        return False
    
    # multiple_boys 568656
    if 568656 in imtags:
        return False
    
    # wiemy że w obrazku nie ma nielegalnych tagów.
    # teraz sprawdzamy czy jest dokładnie jedna postać
    
    #540830 1boy
    #470575 1girl
    return (540830 in imtags) != (470575 in imtags)

In [6]:
def anime_query(images):
    concatdump = set()
    for im in images:
        for t in im.cat_tag[3]:
            concatdump.add(t)
    return [(tagid2tagname[t], t) for t in concatdump]

def imsplit_query(images, anime):
    imdump = []
    for im in images:
        if anime in im.cat_tag[3]:
            imdump.append(im)
            
    return imdump

def succ_characters(ims):
    characters = set()
    for im in ims:
        for tag in im.cat_tag[4]:
            characters |= set([tag])
    return characters

def sift_character_images(images, character_tag):
    cimages = []
    for im in images:
        if character_tag in im.cat_tag[4]:
            cimages.append(im)
    return cimages

def character_check(images, anime):
    if len(images) < 5:
        return False
    occurances = collections.defaultdict(lambda: 0)
    all_ = len(images)
    for im in images:
        for anime_title in im.cat_tag[3]:
            occurances[anime_title] += 1

    ocsrt = list(sorted(list([(float(k[1])/float(all_), k[0]) for k in occurances.items()]), reverse=True))
    for occ, oc in ocsrt:
        if oc == anime:
            return occ > 0.95
    return False
    
        
animes = anime_query(images)
# index 407852
# dragon maid 1394886
def get_anime_character_tags(animeid, images, verbose = False):
    #print(tagid2tagname[animeid])
    images = imsplit_query(images, animeid)
    #print(len(images))
    characters = succ_characters(images)
    character_tags = collections.defaultdict(lambda: None)
            
    for c in characters:
        chartags = collections.defaultdict(lambda: 0)
        imcounts = 0

        sifted_images = sift_character_images(images, c)
        if not character_check(sifted_images, animeid):
            pass
        else:
            #print(tagid2tagname[c], c, len(sifted_images))
            for im in sifted_images:
                if check_for_single_character(im):
                    #print(im.id, [tagid2tagname[k] for k in im.cat_tag[3]])
                    imcounts += 1
                    charactertags = im.cat_tag[0]
                    for tag in charactertags:
                        chartags[tag] += 1

            #print(imcounts)
            tagl = [(k[1], k[0]) for k in chartags.items()]
            tagl = sorted(tagl, reverse=True)
            character_tags[c] = (imcounts, copy.deepcopy(tagl))
            #for c, tid in tagl[:10]:
            #    tp = float(c)/float(imcounts)
            #    print(tp, tagid2tagname[tid], tid)
    return character_tags

#testtags = get_anime_character_tags(407852, images, True)
# obrazki z danego anime


In [9]:
#print(testtags)

def stemmer(word):
    succ = word.split("(")[0]
    if succ[-1] == "_":
        return succ[:-1]
    return succ
    
def chardict_postprocess(resdict):
    characters = resdict.keys()
    joinedcharacter = collections.defaultdict(lambda: [0, collections.defaultdict(lambda: 0)])
    
    for character, chartags in resdict.items():
        if resdict[character]:
            charstem = stemmer(tagid2tagname[character])
            joinedcharacter[charstem][0] += chartags[0]
            for tagcnt, tag in chartags[1]:
                joinedcharacter[charstem][1][tag] += tagcnt

    final_dict = collections.defaultdict(lambda: [0, collections.defaultdict(lambda: 0)])
    for character in joinedcharacter.keys():
        if joinedcharacter[character][0] > 2:
            final_dict[character] = copy.deepcopy(joinedcharacter[character])
    
    return joinedcharacter
            
def charactertags_prettyprinter(tags):
    imcount, tags = tags
    for ittag, itcount in tags.items():
        print(float(itcount)/float(imcount), tagid2tagname[ittag])
    pass

#a = chardict_postprocess(testtags)
#charactertags_prettyprinter(a['saten_ruiko'])

def dump2file(animeid, images):
    #print(f"{tagid2tagname[animeid]}")
    christian_tagname = tagid2tagname[animeid].replace("/", "_")
    try:
        os.mkdir(f"BIG_animedump/{christian_tagname}")
    except Exception as e:
        pass
    
    tags = get_anime_character_tags(animeid, images, False)
    postprocessed = chardict_postprocess(tags)
    if len(postprocessed.keys()) > 0:
        for character in postprocessed.keys():
            christian_character = character.replace("/", "_")
            with open(f"BIG_animedump/{christian_tagname}/{christian_character}.txt", "w") as f:
                f.write(f"{postprocessed[character][0]}\n")
                for tag in postprocessed[character][1].keys():
                    if postprocessed[character][1][tag] > 0:
                        f.write(f"{postprocessed[character][1][tag]}, {tag}, {tagid2tagname[tag]}\n")

pbar = tqdm(animes)
for animename, animeid in pbar:
    pbar.set_description(animename)
    try:
        dump2file(animeid, images)
    except Exception as e:
        print(animename, e)

cafe_little_wish:  51%|█████     | 9245/18266 [1:54:17<2:04:05,  1.21it/s]                                                                                                                                 

monster_hunter string index out of range


moeru!_jiten: 100%|██████████| 18266/18266 [3:37:58<00:00,  1.40it/s]                                                                                                
